In [1]:
import util.yelp as yelp
import numpy as np
from util.text_util import normalize

from hnatt import HNATT

YELP_DATA_PATH = 'data/yelp-dataset/yelp.csv'
SAVED_MODEL_DIR = 'saved_models'
SAVED_MODEL_FILENAME = 'model.h5'
EMBEDDINGS_PATH = 'saved_models/glove.6B.100d.txt'

import pandas as pd

Using TensorFlow backend.


In [2]:
df=pd.read_csv('concatenated_train_pandas.csv')
#df_valid=pd.read_csv('concatenated_dev_pandas.csv')
df_test=pd.read_csv('concatenated_test_pandas.csv')

In [3]:
print(df["x1"][0])
print(df["x2"][0])

Man in a jersey waiting on a crowded subway platform.
the man just came back from a game


In [4]:
df["x1"] = df["x1"] + " " + df["x2"]
#df_valid["x1"] = df_valid["x1"] + " " + df_valid["x2"]
df_test["x1"] = df_test["x1"] + " " + df_test["x2"]


In [5]:
print(df["x1"][0])

Man in a jersey waiting on a crowded subway platform. the man just came back from a game


In [6]:
# convert class labels into class indices
data_classes = ["entails", "neutral", "contradiction"]
df['target']=df['target'].apply(data_classes.index)
#df_valid['target']=df_valid['target'].apply(data_classes.index)
df_test['target']=df_test['target'].apply(data_classes.index)

In [7]:
df=df.dropna()
#df_valid=df_valid.dropna()
df_test=df_test.dropna()

In [8]:
#df=df[:int(df.shape[0]*0.001)]
#df_valid=df_valid[:int(df_valid.shape[0]*0.001)]
#df_test=df_test[:int(df_test.shape[0]*0.001)]

In [9]:
col_text = 'x1'
col_target = 'target'

In [10]:
y_train = df[col_target]
y_test = df_test[col_target]
#y_val = val[col_target].tolist()

In [11]:
print(y_train[4])

1


In [12]:
#train_x=df["x1"]
#train_y=y_train

In [13]:
#train_y.shape

In [14]:
'''
h = HNATT()	
h.train(train_x, train_y, 
batch_size=16,
epochs=16,
embeddings_path=None)

#h.load_weights(SAVED_MODEL_DIR, SAVED_MODEL_FILENAME)



# print attention activation maps across sentences and words per sentence
activation_maps = h.activation_maps(
'they have some pretty interesting things here. i will definitely go back again.')
print(activation_maps)
'''

"\nh = HNATT()\t\nh.train(train_x, train_y, \nbatch_size=16,\nepochs=16,\nembeddings_path=None)\n\n#h.load_weights(SAVED_MODEL_DIR, SAVED_MODEL_FILENAME)\n\n\n\n# print attention activation maps across sentences and words per sentence\nactivation_maps = h.activation_maps(\n'they have some pretty interesting things here. i will definitely go back again.')\nprint(activation_maps)\n"

In [15]:
#(train_x, train_y), (test_x, test_y) = yelp.load_data(path='yelp.csv', size=1e4)

In [16]:
def to_one_hot(labels, dim=5):
	results = np.zeros((len(labels), dim))
	for i, label in enumerate(labels):
		results[i][label - 1] = 1
	return results

In [14]:
df['text_tokens'] = df['x1'].progress_apply(lambda x: normalize(x))
#train_set['len'] = train_set['text_tokens'].apply(lambda x: len(x))

train_x = np.empty((0,))
train_y = np.empty((0,))

training_len=df['x1'].shape[0]

#train_set = df[0:training_len].copy()
#train_set['len'] = train_set['text_tokens'].apply(lambda x: len(x))

train_x=df['text_tokens']
#train_y=train_set['']



train_y = to_one_hot(y_train, dim=3)
print(train_x[0])
print(train_y[0])

#test_y = to_one_hot(test_y)

100%|██████████| 572458/572458 [01:29<00:00, 6390.77it/s]


[u'man in jersey waiting on crowded subway platform', u'man just came back from game']
[1. 0. 0.]


In [17]:
# preprocess test_x , as above block

df_test['text_tokens']=df_test['x1'].progress_apply(lambda x: normalize(x))

#df['text_tokens'] = df['x1'].progress_apply(lambda x: normalize(x))
#train_set['len'] = train_set['text_tokens'].apply(lambda x: len(x))

test_x = np.empty((0,))
test_y = np.empty((0,))

#training_len=df['x1'].shape[0]

#train_set = df[0:training_len].copy()
#train_set['len'] = train_set['text_tokens'].apply(lambda x: len(x))

test_x=df_test['text_tokens']
#train_y=train_set['']



test_y = to_one_hot(y_test, dim=3)
print(test_x[0])
print(test_y[0])



100%|██████████| 11950/11950 [00:03<00:00, 3029.45it/s]

[u'these girls are having great time looking for seashells', u'girls are happy']
[0. 0. 1.]


In [18]:
# load pretrained model
h = HNATT()
h.load_weights(SAVED_MODEL_DIR, SAVED_MODEL_FILENAME)


(None, 50, 100)
(None, 50, 100)
(None, 20, 100)


In [19]:
# test on test set
loss_and_metrics = h.test(test_x, test_y, batch_size=64)
print(loss_and_metrics)

(None, 50, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
word_embeddings (Embedding)  (None, 50, 100)           4467700   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 100)           45300     
_________________________________________________________________
dense_transform_w (Dense)    (None, 50, 100)           10100     
_________________________________________________________________
word_attention (Attention)   (None, 100)               100       
Total params: 4,523,200
Trainable params: 4,523,200
Non-trainable params: 0
_________________________________________________________________
(None, 50, 100)
(None, 20, 100)
_________________________________________________________________
Layer (type)      

In [21]:
h.model.metrics_names

['loss', 'acc']

In [17]:
#h = HNATT()	
h.train(train_x, train_y, 
batch_size=64,
epochs=16,
embeddings_path=None,saved_model_dir=SAVED_MODEL_DIR,
saved_model_filename=SAVED_MODEL_FILENAME)

#h.load_weights(SAVED_MODEL_DIR, SAVED_MODEL_FILENAME)



# print attention activation maps across sentences and words per sentence
#activation_maps = h.activation_maps(
#'they have some pretty interesting things here. i will definitely go back again.')
#print(activation_maps)

(None, 50, 100)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
word_embeddings (Embedding)  (None, 50, 100)           4467700   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 50, 100)           45300     
_________________________________________________________________
dense_transform_w (Dense)    (None, 50, 100)           10100     
_________________________________________________________________
word_attention (Attention)   (None, 100)               100       
Total params: 4,523,200
Trainable params: 4,523,200
Non-trainable params: 0
_________________________________________________________________
(None, 50, 100)
(None, 20, 100)
_________________________________________________________________
Layer (type)      

KeyboardInterrupt: 

In [16]:
activation_maps = h.activation_maps(
'they have some pretty interesting things here. i will definitely go back again.')
print(activation_maps)

[([(u'they', 0.008062019428867473), (u'have', 0.015740954290724567), (u'some', 0.005001129248379999), (u'pretty', 0.39570732830754896), (u'interesting', 0.41721613859206297), (u'things', 0.0911523002322377), (u'here', 0.06712012990017838)], 0.13714904), ([(u'i', 0.03554345484665767), (u'will', 0.044393781401083576), (u'definitely', 0.5791513074525129), (u'go', 0.021449978624936066), (u'back', 0.012777409247274829), (u'again', 0.30668406842753504)], 0.86285096)]


In [24]:
train_y[0]

array([1., 0., 0.])